In [41]:
import yfinance as yf
print(yf.__version__)


0.2.50


In [ ]:
# training.ipynb

import pandas as pd
import numpy as np
import datetime
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import pandas_datareader.data as web


#Load Data (from Stooq)

start = datetime.datetime(2025, 1, 1)
end = datetime.datetime.today()
df=yf.download('GOOG',start=start,end=end)
#df = web.DataReader("AAPL", "stooq", start, end)
df = df.sort_index()   # sort ascending
print("Dataset head:\n", df.head())

# Use only Close prices
data = df[['Close']].values


#  Scale Data

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

# Save scaler for later use
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


# Create Sequences

look_back = 7
x_train, y_train = [], []

for i in range(look_back, len(scaled_data)):
    x_train.append(scaled_data[i-look_back:i, 0])
    y_train.append(scaled_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape for LSTM [samples, timesteps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


# Build Model

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    LSTM(50, return_sequences=False),
    Dense(25),
    Dense(1)
])

model.compile(optimizer="adam", loss="mean_squared_error")


# Train Model

model.fit(x_train, y_train, batch_size=32, epochs=20)


# Save Model

model.save("model.h5")
print("✅ Model and scaler saved successfully!")


Dataset head:
                Open     High      Low    Close     Volume
Date                                                     
2015-01-02  24.7765  24.7882  23.8816  24.3203  239141317
2015-01-05  24.0857  24.1625  23.4499  23.6361  288860738
2015-01-06  23.7270  23.8975  23.2735  23.6381  295587842
2015-01-07  23.8500  24.0639  23.7319  23.9734  180109954
2015-01-08  24.2983  24.9519  24.1754  24.8927  266728489
Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0165
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3.3118e-04
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.0414e-04
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.0123e-04
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.9172e-04
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.0260e-04
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.9059e-04
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.1909e-04
Epoch 9/20
84/84 ━━━━━━━━━━━━━━

✅ Model and scaler saved successfully!
